# Data Analysis results

In [1]:
import numpy as np
from scipy.spatial.distance import squareform
import scipy.cluster.hierarchy as hy
import multiprocessing as mp
from functools import partial
from scipy.stats import entropy
import matplotlib.pyplot as plt
import pickle
from scipy.stats import pearsonr
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
import matplotlib.colors as mcolors
import configparser
config = configparser.ConfigParser()

In [2]:
def calculate_entropies(k):
    """Calculate resolution and relevance from clustering."""
    # resolution
    hs = entropy(k)
    # relevance
    ks = np.unique(k, return_counts=True)
    pk = np.multiply(ks[0], ks[1])
    hk = entropy(pk)
    
    return hs, hk

def cut_and_compute(Z, n):
    """Cut the dendrogram Z in n clusters following maxclust criterion and calculate entropy"""
    cl_labels = hy.fcluster(Z, t=n, criterion='maxclust')
    k=[(cl_labels==t).sum() for t in set(cl_labels)]
       
    res, rel=calculate_entropies(k)
       
    return [res, rel]

def retrive_labels(Z, resrel):
    """Retrive the label vectors corresponding to the maximal resolution"""
    
    Nclu_max = int (np.argmax(resrel[:,1])*10+1)
    label= hy.fcluster(Z, t=Nclu_max, criterion='maxclust')
    
    return label


def ResRel(distance_matrix):
 
    dist=distance_matrix
    dist_max=dist[dist!=np.inf].max()

    sdist=squareform(dist/dist_max, checks=False)

    M=int((1+np.sqrt(1+8*sdist.shape[0]))/2) 
    Nclu=range(1,M,10) 
    
       
    ## Compute the dendrogram
    Z = hy.linkage(sdist,'average')
    
    ## start the computation in parallel
    
    pool = mp.Pool(10)
    temp = partial( cut_and_compute , Z )
    result = pool.map ( func = temp , iterable = Nclu )
    pool.close()
    pool.join()

    label = retrive_labels(Z, np.array(result))

    return np.array(result), label, M

In [3]:
def calculate_centroid(cluster_indices, dist_matrix):
    # Select rows of the RMSD matrix
    sub_matrix = dist_matrix[cluster_indices]
    # Compute the controid (mean of the row)
    centroid = sub_matrix.mean(axis=0)
    return centroid


def Cluster_Distance(label_vector, rmsd_matrix, method='average'):
    num_clusters = np.max(label_vector)
    distance_matrix = np.zeros((num_clusters, num_clusters))

    for i in range(1, num_clusters + 1):
        for j in range(i, num_clusters + 1):
            # Find indeces of configurations in cluster i and j
            cluster_i_indices = np.where(label_vector == i)[0]
            cluster_j_indices = np.where(label_vector == j)[0]

            if i != j:
                if method == 'single':
                    # Single linkage: minimum distance between cluster members
                    distance = rmsd_matrix[np.ix_(cluster_i_indices, cluster_j_indices)].min()
                elif method == 'complete':
                    # Complete linkage: maximum distance between cluster members
                    distance = rmsd_matrix[np.ix_(cluster_i_indices, cluster_j_indices)].max()
                elif method == 'average':
                    # Average linkage: avarage distance between cluster members
                    distance = rmsd_matrix[np.ix_(cluster_i_indices, cluster_j_indices)].mean()
                elif method == 'centroid':
                    # Centroid linkage: distance between cluster centroids
                    centroid_i = calculate_centroid(cluster_i_indices, rmsd_matrix)
                    centroid_j = calculate_centroid(cluster_j_indices, rmsd_matrix)
                    distance = np.linalg.norm(centroid_i - centroid_j)
                elif method == 'ward':
                    # Ward linkage: minimize total variance
                    centroid_i = calculate_centroid(cluster_i_indices, rmsd_matrix)
                    centroid_j = calculate_centroid(cluster_j_indices, rmsd_matrix)
                    distance = np.sum((centroid_i - centroid_j) ** 2)
            else:
                distance = 0  # Distance between a cluster and itself is zero

            # Load the distance matrix
            distance_matrix[i - 1, j - 1] = distance
            distance_matrix[j - 1, i - 1] = distance 

    return distance_matrix

## Import data obtained from analysis

In [4]:
config.read('par_dist.dat')

input_gro = config['DEFAULT']['frame']
input_xtc = config['DEFAULT']['traj']
sel = config['DEFAULT']['selection']
metrics = config['DEFAULT']['metrics'].split(', ')
strides_names = config['DEFAULT']['strides'].split(', ')

strides = [int(num) for num in strides_names]

metrics_dict = {
    'wrmsd': 'WRMSD',
    'rmsd_L1': f'RMSD $L^1$',
    'rmsd_L2': f'RMSD $L^2$',
    'rmsd_Linf': 'RMSD $L^{\infty}$',
    'Rgyr' : f'R$_g$',
    'CMAP_H': 'CMAP$_H$',
    'CMAP_J': 'CMAP$_J$',
    'CMAP_cos': 'CMAP$_{cos}$'
}

names = [metrics_dict[metric] for metric in metrics if metric in metrics_dict]

for metric in metrics:
    if metric not in metrics_dict:
        raise ValueError(f"Error: Metric '{metric}' is NOT defined.")

if sel == 'name CA':
    save_path = 'save_and_load/C_A'
if sel == 'all':
    save_path = 'save_and_load/AA'

In [5]:
distances = []

for met in metrics:
    mat = np.load(save_path+f'/distances/{met}.npy')
    distances.append(mat)

## Resolution and Relevance curves

In [6]:
with open(save_path+'/ResRel/ResRel_UNI.pkl', 'rb') as file:
    ResRel_UNI = pickle.load(file)

with open(save_path+'/ResRel/labels_UNI.pkl', 'rb') as file:
    labels_UNI = pickle.load(file)

with open(save_path+'/ResRel/ResRel_RAND.pkl', 'rb') as file:
    ResRel_RAND = pickle.load(file)

with open(save_path+'/ResRel/labels_RAND.pkl', 'rb') as file:
    labels_RAND = pickle.load(file)

with open(save_path+'/ResRel/M.pkl', 'rb') as file:
    M = pickle.load(file)

In [7]:
MSR_uniform = []

for j in range(len(metrics)):

    msr_values = []

    for i in range(len(strides)):
        m = M[i]
    
        label = labels_UNI[i][j]
        resrel = ResRel_UNI[i][j]

        x = resrel[:, 0] / np.log(m)
        y = resrel[:, 1] / np.log(m)
        MSR = np.trapz(y,x)
        msr_values.append(MSR)

    MSR_uniform.append(msr_values)

MSR_random = []

for j in range(len(metrics)):

    msr_values = []
    for i in range(len(strides)):
        m = M[i]
    
        label = labels_RAND[i][j]
        resrel = ResRel_RAND[i][j]

        x = resrel[:, 0] / np.log(m)
        y = resrel[:, 1] / np.log(m)
        MSR = np.trapz(y,x)
        msr_values.append(MSR)

    MSR_random.append(msr_values)

In [ ]:
colors = plt.cm.tab10.colors

plt.figure(figsize=(18, 5))
for i in range(len(metrics)):
    color = colors[i % len(colors)]
    plt.plot(MSR_uniform[i], marker='o', label=names[i], c=color)
    plt.plot(MSR_random[i], marker='*', c=color)

plt.xlabel('Stride', fontsize=20)
plt.title('MSR', fontsize=30, pad=15)
plt.legend()
xticks = range(len(strides))
xticks_labels = strides
plt.xticks(ticks=xticks, labels=xticks_labels)
plt.grid(True, linestyle=':', color='gray', alpha=0.7)
plt.show()
print('POINT: Uniform stride    STAR: Random stride')

## Inter/Intra-Cluster Covariance matrices traces

In [9]:
with open(save_path+'/COV_MAT/INTER_uniform.pkl', 'rb') as file:
    INTER_uniform = pickle.load(file)

with open(save_path+'/COV_MAT/INTRA_uniform.pkl', 'rb') as file:
    INTRA_uniform = pickle.load(file)

with open(save_path+'/COV_MAT/INTER_random.pkl', 'rb') as file:
    INTER_random = pickle.load(file)

with open(save_path+'/COV_MAT/INTRA_random.pkl', 'rb') as file:
    INTRA_random = pickle.load(file)

In [ ]:
colors = plt.cm.tab10.colors

plt.figure(figsize=(18, 5))
for i in range(len(metrics)):
    color = colors[i % len(colors)]
    plt.plot(INTER_uniform[i], marker='o', label=names[i], c=color)  # Label dinamico per ciascuna metrica
    plt.plot(INTER_random[i], marker='*', c=color)

# Aggiungi etichette agli assi
plt.xlabel('Stride', fontsize=15)

# Aggiungi titolo
plt.title('Inter-cluster cov matrix trace',fontsize=25, pad=15)

# Aggiungi la legenda
plt.legend()

# Imposta i ticks sull'asse x (se vuoi che siano esattamente uguali a quelli di 'strides')
xticks = range(len(strides))
xticks_labels = strides
plt.xticks(ticks=xticks, labels=xticks_labels)

# Mostra il grafico
plt.grid(True, linestyle=':', color='gray', alpha=0.7)
plt.show()

In [ ]:
colors = plt.cm.tab10.colors

plt.figure(figsize=(18, 5))
for i in range(len(metrics)):
    color = colors[i % len(colors)]
    plt.plot(INTRA_uniform[i], marker='o', label=names[i], c=color)  # Label dinamico per ciascuna metrica
    plt.plot(INTRA_random[i], marker='*', c=color)

# Aggiungi etichette agli assi
plt.xlabel('Stride')

# Aggiungi titolo
plt.title('Intra-cluster cov matrix trace',fontsize=25, pad=15)

# Aggiungi la legenda
plt.legend()

# Imposta i ticks sull'asse x (se vuoi che siano esattamente uguali a quelli di 'strides')
xticks = range(len(strides))
xticks_labels = strides
plt.xticks(ticks=xticks, labels=xticks_labels)

# Mostra il grafico
plt.grid(True, linestyle=':', color='gray', alpha=0.7)
plt.show()

## Diffusion Maps

Now let's compute the Diffusion Maps for out dataset. We consider here only the case without striding. To see computational details, the runned code is commented in the cell below

For a practical manipulation of datas, this has been runned once, saving the data on an external files which now we can re-load

In [12]:
with open(save_path+'/DMAPS/HR_DC1.pkl', 'rb') as file:
    HR_DC1 = pickle.load(file)

with open(save_path+'/DMAPS/HR_DC2.pkl', 'rb') as file:
    HR_DC2 = pickle.load(file)

with open(save_path+'/DMAPS/LR_DC1.pkl', 'rb') as file:
    LR_DC1 = pickle.load(file)

with open(save_path+'/DMAPS/LR_DC2.pkl', 'rb') as file:
    LR_DC2 = pickle.load(file)

def rescale_to_unit_interval(values):
    min_val = np.min(values)
    max_val = np.max(values)
    return 2 * (values - min_val) / (max_val - min_val) - 1

# NORMALIZATION OF THE LOADED DIFFUSION COORDINATES (+ Some constant factors to compare the plots)

x1 = []
y1 = []
x2 = []
y2 = []

facx1 = [-1, 1, 1, 1, 1, 1, -1, 1]
facy1 = [1, 1, -1, 1, 1, -1, -1, -1]
facx2 = [1, -1, 1, -1, -1, 1, -1, 1]
facy2 = [1, -1, 1, -1, 1, -1, 1, 1]

for i in range(len(HR_DC1)):
    x1.append(facx1[i]*rescale_to_unit_interval(HR_DC1[i]))
    y1.append(facy1[i]*rescale_to_unit_interval(HR_DC2[i]))
    x2.append(facx2[i]*rescale_to_unit_interval(LR_DC1[i]))
    y2.append(facy2[i]*rescale_to_unit_interval(LR_DC2[i]))

In [ ]:
for i in range(len(HR_DC1)):

    fig, axes=plt.subplots(1,2, figsize=(15,6), sharex=True, sharey=True)

    norm = mcolors.Normalize(vmin=x2[i].min(), vmax=x2[i].max())
    colormap = plt.cm.viridis # Scegli la colormap viridis
    colors = colormap(norm(x2[i]))

    # Scatter plot per 'x2' e 'y2' con il gradiente di colori basato su x2
    p = axes[1].scatter(x2[i], y2[i], c=x2[i], cmap=colormap, norm=norm) 

    # Colormap per il primo scatter plot
    color_idx = [x2[i][ii-1] for ii in labels_UNI[0][i]]
    pw = axes[0].scatter(x1[i], y1[i], c=color_idx, cmap=colormap, norm=norm)

    plt.colorbar(pw,ax=axes[0])
    plt.colorbar(p,ax=axes[1])

    axes[0].set_xlabel('1st Diffusion Coordinate',fontsize=15)
    axes[0].set_ylabel('2nd Diffusion Coordinate',fontsize=15)
    axes[1].set_xlabel('1st Diffusion Coordinate',fontsize=15)

    axes[0].tick_params(axis="both", labelsize=13)
    axes[1].tick_params(axis="both", labelsize=13)

    fig.suptitle(names[i],fontsize=25, y=1.0)
    plt.tight_layout()
    plt.show()

In [ ]:
from itertools import combinations
import math

combinazioni = [(i, j) for i, j in combinations(range(len(x1)), 2)]

# Calcola il numero di righe e colonne in base al numero di combinazioni
num_plots = len(combinazioni)
cols = 3  # Numero di colonne fisso (puoi cambiarlo se vuoi)
rows = math.ceil(num_plots / cols)  # Numero di righe necessario per visualizzare tutte le combinazioni

# Creazione della griglia per i subplot quadrati
fig, axes = plt.subplots(rows, cols, figsize=(6 * cols, 6 * rows))

# Aggiungi ogni combinazione (i, j) a un subplot
for idx, (i, j) in enumerate(combinazioni):
    row = idx // cols
    col = idx % cols
    ax = axes[row, col] if rows > 1 else axes[col]  # Gestisce sia griglie 1D che 2D
    ax.scatter(x1[i], x1[j], marker='.', c='darkred')
    ax.set_xlabel(names[i],fontsize=15)
    ax.set_ylabel(names[j],fontsize=15)

# Rimuove eventuali subplot vuoti
for idx in range(num_plots, rows * cols):
    fig.delaxes(axes.flatten()[idx])

plt.suptitle("Scatterplots of the First DC",fontsize=25, y=1.0)
plt.tight_layout()
plt.show()

In [ ]:
Conf_MAT = np.zeros((len(x1), len(x1)))

for i in range(len(x1)):
    for j in range(len(x1)):
        correlation, p_value = pearsonr(x1[i], x1[j])
        Conf_MAT[i, j] = correlation


plt.imshow(Conf_MAT, cmap="coolwarm", aspect="auto")
plt.colorbar()
plt.xticks(ticks=np.arange(len(names)), labels=names, rotation=45, fontsize=12)
plt.yticks(ticks=np.arange(len(names)), labels=names, fontsize=12)
plt.title(f"PCC between DC1", fontsize=25, pad=20)

for i in range(Conf_MAT.shape[0]):
    for j in range(Conf_MAT.shape[1]):
        plt.text(j, i, f"{Conf_MAT[i, j]:.3f}", ha="center", va="center", fontsize=11)


# Adjusted Rand Index & Adjusted Mutual Information

In [14]:
ARI = np.zeros((len(x1), len(x1)))
AMI = np.zeros((len(x1), len(x1)))

for i in range(len(x1)):
    for j in range(len(x1)):
    
        ARI[i, j] = adjusted_rand_score(labels_UNI[0][i], labels_UNI[0][j])
        AMI[i, j] = adjusted_mutual_info_score(labels_UNI[0][i], labels_UNI[0][j])

In [ ]:
# 1st heatmap
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.imshow(ARI, cmap="coolwarm", aspect="auto",vmin=0,vmax=1)
plt.colorbar()
plt.xticks(ticks=np.arange(len(names)), labels=names, rotation=45, fontsize=12)
plt.yticks(ticks=np.arange(len(names)), labels=names, fontsize=12)
plt.title("Adjusted Rand Index", fontsize=25, pad=20)

for i in range(ARI.shape[0]):
    for j in range(ARI.shape[1]):
        plt.text(j, i, f"{ARI[i, j]:.3f}", ha="center", va="center", fontsize=11)


# 2nd heatmap
plt.subplot(1, 2, 2)
plt.imshow(AMI, cmap="coolwarm", aspect="auto",vmin=0,vmax=1)
plt.colorbar()
plt.xticks(ticks=np.arange(len(names)), labels=names, rotation=45, fontsize=12)
plt.yticks(ticks=np.arange(len(names)), labels=names, fontsize=12)
plt.title("Adjusted Mutual Information", fontsize=25, pad=20)

for i in range(AMI.shape[0]):
    for j in range(AMI.shape[1]):
        plt.text(j, i, f"{AMI[i, j]:.3f}", ha="center", va="center", fontsize=11)

plt.tight_layout()
plt.show()

# Coarse Grained Trajectories: different metrics

In [ ]:
indeces = [2, 4, 6]  # Write here the indeces of the metrics you are interested in plotting


for i in range(len(indeces)):
    
    ind = indeces[i]
    #others = indeces[:i] + indeces[i+1:] 
    others = indeces

    plt.figure(figsize=(18, 6))
    plt.plot(distances[ind][0], label='Full traj', marker='none')

    for j in indeces:

        clusters = np.unique(labels_UNI[0][j])
        cluster_means = {cluster: distances[ind][0][labels_UNI[0][j] == cluster].mean() for cluster in clusters}
        expanded_means = np.array([cluster_means[label] for label in labels_UNI[0][j]])
        plt.plot(expanded_means, label=f'{names[j]} clustering', marker='none')

    plt.xlabel('Frame', fontsize=15)
    plt.title(names[ind], fontsize=30, pad=15)
    plt.legend()
    plt.grid(True, linestyle=':', color='gray', alpha=0.7)

In [ ]:
n = len(indeces)
Nbins = 70
window_size = 10  # Frames counted for the moving avarage

fig,ax = plt.subplots(n, n+1, figsize=(20,15))

for i in range(len(indeces)):
    ind = indeces[i]
    others = indeces
    data = distances[ind][0]
    
    moving_avg = np.convolve(data, np.ones(window_size) / window_size, mode='valid')

    # FIRST PLOT: Moving avarage
    ax[i, 0].hist(data, label='Full_traj', bins=Nbins, log=True)
    ax[i, 0].hist(moving_avg, label='Moving avg (50 frames)', bins=Nbins, alpha=0.6, range=(data.min(), data.max()), log=True)
    ax[i, 0].set_title(names[ind], fontsize=20, pad=15)
    ax[i, 0].set_xlabel('Frame', fontsize=15)
    ax[i, 0].legend()
    ax[i, 0].grid(linestyle=':', color='gray', alpha=0.5, zorder=0)

    for idx, j, in enumerate(others):

        ax[i,idx +1].hist(data, label='Full traj', bins=Nbins, log=True)

        clusters = np.unique(labels_UNI[0][j])
        cluster_means = {cluster: data[labels_UNI[0][j] == cluster].mean() for cluster in clusters}
        expanded_means = np.array([cluster_means[label] for label in labels_UNI[0][j]])
        ax[i,idx +1].hist(expanded_means, label=f'{names[j]} clustering', bins=Nbins, alpha=0.6, range=(data.min(),data.max()), log=True)

        ax[i,idx +1].set_xlabel('Frame', fontsize=15)
        ax[i,idx +1].set_title(names[ind], fontsize=20, pad=15)
        ax[i,idx +1].legend()
        ax[i, idx +1].grid(linestyle=':', color='gray', alpha=0.5, zorder=0)

# IF YOU WANT TO CUT A ROW JUST RUN:
for col in range(n+1):
    fig.delaxes(ax[2, col])

plt.tight_layout()

# PCA and RMSIP

We want to look at the first principal components of the full trajectory and the coarse grained trajectories obtained by the highest-relavance clustering found by Resolution and Relevance.

The idea is to compare the low-dimensional subspaces spanned by the first $S$ Principal Components, looking at how similar they are.

To quantify this similarity we use the Root Mean Square Inner Product (RMSIP), which is defined as:
$$
RMSIP(\{ \vec{u}_i \}_{i=1,..,S}, \{ \vec{v}_j \}_{j=1,..,S}) = \sqrt{\frac{1}{S} \sum_{i} \sum_{j} (\vec{u}_i \cdot \vec{v}_j)^2}
$$

The RMSIP value varies between 0 and 1: 0 indicates that the subspaces are mutually orthogonal, whereas 1 indicates that they are identical.

In [20]:
def RMSIP(set1, set2):
    """
    Compute the RMSIP (Root Mean Square Inner Product) between two sets of orthonormal basis vectors.
    
    Parameters:
        set1 (numpy.ndarray): A (n x k) matrix where each column is a basis vector (orthonormalized).
        set2 (numpy.ndarray): A (n x k) matrix where each column is a basis vector (orthonormalized).
    
    Returns:
        float: The RMSIP value.
    """
    # Ensure input is numpy array
    set1 = np.array(set1)
    set2 = np.array(set2)
    
    # Validate dimensions
    if set1.shape != set2.shape:
        raise ValueError("The two sets must have the same dimensions (n x k).")
    
    # Compute the pairwise inner products
    inner_product_matrix = np.dot(set1.T, set2)
    
    # Square the elements of the inner product matrix
    squared_inner_products = inner_product_matrix ** 2
    
    # Compute RMSIP
    k = set1.shape[1]  # Number of basis vectors
    rmsip = np.sqrt(np.sum(squared_inner_products) / k)
    
    return rmsip

In [ ]:
PCs = []

full_PCs = np.load(save_path+'/PCA/PCs_full_traj.npy')
N_PCs = 4  # Number of PCs compared by RMSIP, it must be < N_frames

sub_matrix = full_PCs[:, :N_PCs]
PCs.append(sub_matrix)


for met in metrics:
    clustered_PCs = np.load(save_path+f'/PCA/PCs_{met}.npy')
    sub_matrix = clustered_PCs[:, :N_PCs]
    PCs.append(sub_matrix)

RMSIP_MAT = np.zeros((len(PCs), len(PCs)))

for i in range(len(RMSIP_MAT)):
    for j in range(len(RMSIP_MAT)):
        RMSIP_MAT[i, j] = RMSIP(PCs[i],PCs[j])


plt.imshow(RMSIP_MAT, cmap="coolwarm", aspect="auto")
plt.colorbar()
plt.xticks(ticks=np.arange(len(names)+1), labels=['full']+names, rotation=45, fontsize=12)
plt.yticks(ticks=np.arange(len(names)+1), labels=['full']+names, fontsize=12)
plt.title(f"RMSIP between first {N_PCs} PCs", fontsize=25, pad=20)

for i in range(RMSIP_MAT.shape[0]):
    for j in range(RMSIP_MAT.shape[1]):
        plt.text(j, i, f"{RMSIP_MAT[i, j]:.3f}", ha="center", va="center", fontsize=11)